# 환경 설정

In [1]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.2 MB/s eta 0:00:00


In [ ]:
!rm -rf /content/YOLO_safety
print("🧹 YOLO_safety 폴더 완전 삭제 완료")

🧹 YOLO_safety 폴더 완전 삭제 완료


In [7]:
!rm -rf /content/YOLO_safety/val

# 데이터 압축 해제

In [3]:
# zip 파일 읽어서 압축 해제

import os
import zipfile

# 경로 설정
base_dir = "/content/YOLO_safety"   # colab 로컬 디렉토리
os.makedirs(base_dir, exist_ok=True)

zip_path = "/content/drive/MyDrive/AI/"

# 압축 해제 함수
def unzip_files(zip_path, target_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir)
    print(f"압축 해제 완료: {zip_path} -> {target_dir}")

# 압축 해제
file_name = "final.zip"
unzip_files(zip_path + file_name, base_dir)

# # 압축 해제
# file_name = "2_bbox_completed.zip"
# unzip_files(zip_path + file_name, base_dir)

압축 해제 완료: /content/drive/MyDrive/AI/final.zip -> /content/YOLO_safety


# JSON 라벨을 YOLO 포맷으로 변환

In [4]:
import json
from pathlib import Path

def convert_box_json_to_yolo(json_dir, output_dir, class_map, log_path="convert_log.txt"):
    """
    box 또는 polygon 기반 JSON 라벨을 YOLO 포맷으로 변환하는 함수.
    - polygon은 좌표의 min/max로 bounding box 추정
    - bbox 유효성 검사 포함
    - 객체가 없더라도 빈 .txt 파일을 생성하여 학습 누락 방지
    """
    json_dir = Path(json_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    log_file = Path(log_path)

    json_files = list(json_dir.glob("*.json"))
    log_lines = [f"총 {len(json_files)}개의 JSON 파일을 처리합니다.\n\n"]

    converted_count = 0
    skipped_empty = 0
    skipped_class = 0
    skipped_invalid_bbox = 0

    for json_file in json_files:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        filename = data["image"]["filename"]
        img_width, img_height = data["image"]["resolution"]
        label_lines = []

        for ann in data.get("annotations", []):
            class_id_raw = ann.get("class")
            if class_id_raw not in class_map:
                skipped_class += 1
                continue

            class_id = class_map[class_id_raw]

            # box 우선 사용
            box = ann.get("box")
            if box and len(box) == 4:
                xmin, ymin, xmax, ymax = box
            elif "polygon" in ann and len(ann["polygon"]) >= 3:
                xs = [pt[0] for pt in ann["polygon"]]
                ys = [pt[1] for pt in ann["polygon"]]
                xmin, xmax = min(xs), max(xs)
                ymin, ymax = min(ys), max(ys)
            else:
                continue

            if (xmax - xmin < 1) or (ymax - ymin < 1):
                skipped_invalid_bbox += 1
                continue
            if not (0 <= xmin < xmax <= img_width) or not (0 <= ymin < ymax <= img_height):
                skipped_invalid_bbox += 1
                continue

            # YOLO format 변환
            cx = ((xmin + xmax) / 2) / img_width
            cy = ((ymin + ymax) / 2) / img_height
            w = (xmax - xmin) / img_width
            h = (ymax - ymin) / img_height

            label_lines.append(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

        label_file = output_dir / (Path(filename).stem + ".txt")
        with open(label_file, "w") as out_f:
            if label_lines:
                out_f.write("\n".join(label_lines))
                converted_count += 1
                log_lines.append(f"✅ 변환 완료: {filename} → {len(label_lines)}개 객체\n")
            else:
                out_f.write("")  # 객체 없더라도 빈 파일 생성
                skipped_empty += 1
                log_lines.append(f"⚠️ 객체 없음 (빈 라벨 생성): {filename}\n")

    # 최종 요약
    log_lines.append("\n📌 변환 요약 📌\n")
    log_lines.append(f"✅ 변환된 라벨 수: {converted_count}개\n")
    log_lines.append(f"⚠️ 빈 라벨 (객체 없음): {skipped_empty}개\n")
    log_lines.append(f"🚫 class_map 미등록 클래스: {skipped_class}개\n")
    log_lines.append(f"🚫 잘못된 bbox (너무 작거나 범위 초과): {skipped_invalid_bbox}개\n")

    # 로그 파일 저장 (마지막에 한 번만 엶)
    with log_file.open("w", encoding="utf-8") as log:
        log.writelines(log_lines)

    # 이미지 디렉토리 존재 확인
    print("✅ train images exists:", Path("/content/YOLO_safety/train/images").exists())
    print("✅ val images exists:", Path("/content/YOLO_safety/val/images").exists())

    # 라벨 디렉토리 + 파일 개수 확인
    train_labels = list(Path("/content/YOLO_safety/train/labels").glob("*.txt"))
    print(f"✅ train 라벨 수: {len(train_labels)}개")

    val_labels = list(Path("/content/YOLO_safety/val/labels").glob("*.txt"))
    print(f"✅ val 라벨 수: {len(val_labels)}개")

    # 비어있는 파일 확인
    empty = [f for f in val_labels if f.stat().st_size == 0]
    print(f"⚠️ 비어있는 라벨 파일 수: {len(empty)}개")

    print(f"🔍 라벨 변환 로그가 '{log_path}'에 저장되었습니다.")


In [8]:
# 클래스 매핑: JSON 클래스 번호(str) → YOLO 클래스 인덱스(int)
class_mapping_all = {
    "01": 0,  # 안전벨트 착용
    "07": 1,  # 안전모 착용
}

# ✅ 학습 데이터 전체 변환 (01, 07 모두 포함된 json 폴더 기준)
convert_box_json_to_yolo(
    json_dir="/content/YOLO_safety/train/labels_json",  # 통합 디렉토리
    output_dir="/content/YOLO_safety/train/labels",
    class_map=class_mapping_all,
    log_path="train_log.txt"
)
print("✅ 학습 데이터 전체 라벨 변환 완료")

# ✅ 검증 데이터 전체 변환
convert_box_json_to_yolo(
    json_dir="/content/YOLO_safety/val/labels_json",  # 통합 디렉토리
    output_dir="/content/YOLO_safety/val/labels",
    class_map=class_mapping_all,
    log_path="val_log.txt"
)
print("✅ 검증 데이터 전체 라벨 변환 완료")


✅ train images exists: True
✅ val images exists: True
✅ train 라벨 수: 44554개
✅ val 라벨 수: 0개
⚠️ 비어있는 라벨 파일 수: 0개
🔍 라벨 변환 로그가 'train_log.txt'에 저장되었습니다.
✅ 학습 데이터 전체 라벨 변환 완료
✅ train images exists: True
✅ val images exists: True
✅ train 라벨 수: 44554개
✅ val 라벨 수: 10820개
⚠️ 비어있는 라벨 파일 수: 0개
🔍 라벨 변환 로그가 'val_log.txt'에 저장되었습니다.
✅ 검증 데이터 전체 라벨 변환 완료


In [11]:
from pathlib import Path
import os

labels_dir = Path("/content/YOLO_safety/train/labels")  # 라벨 폴더 경로 수정

for txt_file in labels_dir.glob("*.txt"):
    new_name = txt_file.with_name(txt_file.stem.lower() + ".txt")
    os.rename(txt_file, new_name)
    print(f"Renamed: {txt_file.name} → {new_name.name}")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Renamed: S2-N1106M10286.txt → s2-n1106m10286.txt
Renamed: S2-N1206M02248.txt → s2-n1206m02248.txt
Renamed: S2-N1105M01945.txt → s2-n1105m01945.txt
Renamed: S2-N1106M10030.txt → s2-n1106m10030.txt
Renamed: S2-N1106M24697.txt → s2-n1106m24697.txt
Renamed: S2-N1106M05679.txt → s2-n1106m05679.txt
Renamed: S2-N1106M02600.txt → s2-n1106m02600.txt
Renamed: S2-N1001M01557.txt → s2-n1001m01557.txt
Renamed: S2-N1106M22564.txt → s2-n1106m22564.txt
Renamed: S2-N1106M18912.txt → s2-n1106m18912.txt
Renamed: S2-N1106M16778.txt → s2-n1106m16778.txt
Renamed: S2-N1206M00785.txt → s2-n1206m00785.txt
Renamed: S2-N1106M21056.txt → s2-n1106m21056.txt
Renamed: S2-N1402M03340.txt → s2-n1402m03340.txt
Renamed: S2-N1001M01363.txt → s2-n1001m01363.txt
Renamed: S2-N1106M08485.txt → s2-n1106m08485.txt
Renamed: S2-N1106M03883.txt → s2-n1106m03883.txt
Renamed: S2-N1106M05399.txt → s2-n1106m05399.txt
Renamed: S2-N1106M14397.txt → s2-n1106m14397.txt
Renamed: S2-O1301M00083.txt → s2-

In [ ]:
from pathlib import Path
import os

# 이미지 디렉토리 경로 지정
train_images_dir = Path("/content/YOLO_safety/train/images")

def rename_to_lowercase(directory):
    for img_file in directory.glob("*.*"):
        lower_name = img_file.with_name(img_file.stem.lower() + img_file.suffix.lower())
        if img_file != lower_name:  # 이름이 다를 때만 실행
            os.rename(img_file, lower_name)
            print(f"Renamed: {img_file.name} → {lower_name.name}")

# 실행
print("📂 Train 이미지 파일명 수정:")
rename_to_lowercase(train_images_dir)



📂 Train 이미지 파일명 수정:
Renamed: S2-N1105M00015.jpg → s2-n1105m00015.jpg
Renamed: S2-N1104M02215.jpg → s2-n1104m02215.jpg
Renamed: S2-N1106M03219.jpg → s2-n1106m03219.jpg
Renamed: S2-N1104M01733.jpg → s2-n1104m01733.jpg
Renamed: S2-N1106M04211.jpg → s2-n1106m04211.jpg
Renamed: S2-N1106M04103.jpg → s2-n1106m04103.jpg
Renamed: S2-N1104M02640.jpg → s2-n1104m02640.jpg
Renamed: S2-N1105M02558.jpg → s2-n1105m02558.jpg
Renamed: S2-N1105M07449.jpg → s2-n1105m07449.jpg
Renamed: S2-N1105M05590.jpg → s2-n1105m05590.jpg
Renamed: S2-N1106M08647.jpg → s2-n1106m08647.jpg
Renamed: S2-N1106M07205.jpg → s2-n1106m07205.jpg
Renamed: S2-N1106M10194.jpg → s2-n1106m10194.jpg
Renamed: S2-N1105M12347.jpg → s2-n1105m12347.jpg
Renamed: S2-N1105M20793.jpg → s2-n1105m20793.jpg
Renamed: S2-N1106M04045.jpg → s2-n1106m04045.jpg
Renamed: S2-O1301M02942.jpg → s2-o1301m02942.jpg
Renamed: S2-N1106M10219.jpg → s2-n1106m10219.jpg
Renamed: S2-N1105M20837.jpg → s2-n1105m20837.jpg
Renamed: S2-N1105M08080.jpg → s2-n1105m08080.jpg


In [14]:
from pathlib import Path

# 디렉토리 설정
train_image_dir = Path("/content/YOLO_safety/train/images")
val_image_dir = Path("/content/YOLO_safety/val/images")

train_label_dir = Path("/content/YOLO_safety/train/labels")
val_label_dir = Path("/content/YOLO_safety/val/labels")

def check_dataset(image_dir, label_dir, dataset_name="Dataset"):
    images = list(image_dir.glob("*.jpg")) + list(image_dir.glob("*.png"))
    labels = list(label_dir.glob("*.txt"))

    image_names = {img.stem for img in images}
    label_names = {lbl.stem for lbl in labels}

    # 매칭 검사
    no_label = image_names - label_names  # 라벨 없는 이미지
    no_image = label_names - image_names  # 이미지 없는 라벨

    print(f"\n📂 {dataset_name} 데이터셋 확인")
    print(f" - 이미지 수: {len(images)}개")
    print(f" - 라벨 수: {len(labels)}개")

    if no_label:
        print(f" ⚠️ 라벨 없는 이미지 {len(no_label)}개 예시: {list(no_label)[:3]}")
    else:
        print(" ✅ 모든 이미지에 라벨이 존재")

    if no_image:
        print(f" ⚠️ 이미지 없는 라벨 {len(no_image)}개 예시: {list(no_image)[:3]}")
    else:
        print(" ✅ 모든 라벨에 대응되는 이미지 존재")

# 학습 / 검증 데이터셋 확인
check_dataset(train_image_dir, train_label_dir, "학습 (Train)")
check_dataset(val_image_dir, val_label_dir, "검증 (Val)")

# 총합
train_count = len(list(train_label_dir.glob("*.txt")))
val_count = len(list(val_label_dir.glob("*.txt")))
print(f"\n📊 총 라벨 파일 개수: {train_count + val_count}개 (학습 {train_count}개 + 검증 {val_count}개)")



📂 학습 (Train) 데이터셋 확인
 - 이미지 수: 43284개
 - 라벨 수: 43284개
 ✅ 모든 이미지에 라벨이 존재
 ✅ 모든 라벨에 대응되는 이미지 존재

📂 검증 (Val) 데이터셋 확인
 - 이미지 수: 10820개
 - 라벨 수: 10820개
 ✅ 모든 이미지에 라벨이 존재
 ✅ 모든 라벨에 대응되는 이미지 존재

📊 총 라벨 파일 개수: 54104개 (학습 43284개 + 검증 10820개)


In [13]:
from pathlib import Path
import os

# 디렉토리 설정
train_image_dir = Path("/content/YOLO_safety/train/images")
val_image_dir = Path("/content/YOLO_safety/val/images")

train_label_dir = Path("/content/YOLO_safety/train/labels")
val_label_dir = Path("/content/YOLO_safety/val/labels")

# JSON 경로 설정 (YOLO 라벨 json 폴더가 있다면)
train_json_dir = Path("/content/YOLO_safety/train/labels_json")
val_json_dir = Path("/content/YOLO_safety/val/labels_json")

def clean_unmatched(image_dir, label_dir, json_dir=None, dataset_name="Dataset"):
    images = list(image_dir.glob("*.jpg")) + list(image_dir.glob("*.png"))
    labels = list(label_dir.glob("*.txt"))

    image_names = {img.stem for img in images}
    label_names = {lbl.stem for lbl in labels}

    # 매칭 검사
    no_label = image_names - label_names  # 라벨 없는 이미지
    no_image = label_names - image_names  # 이미지 없는 라벨

    print(f"\n🧹 {dataset_name} 데이터셋 정리 시작")

    # 라벨 없는 이미지 삭제
    for stem in no_label:
        for ext in [".jpg", ".png"]:
            img_path = image_dir / f"{stem}{ext}"
            if img_path.exists():
                os.remove(img_path)
                print(f"🗑️ 이미지 삭제: {img_path}")

        if json_dir:  # JSON도 같이 삭제
            json_path = json_dir / f"{stem}.json"
            if json_path.exists():
                os.remove(json_path)
                print(f"🗑️ JSON 삭제: {json_path}")

    # 이미지 없는 라벨 삭제
    for stem in no_image:
        txt_path = label_dir / f"{stem}.txt"
        if txt_path.exists():
            os.remove(txt_path)
            print(f"🗑️ 라벨 삭제: {txt_path}")

        if json_dir:
            json_path = json_dir / f"{stem}.json"
            if json_path.exists():
                os.remove(json_path)
                print(f"🗑️ JSON 삭제: {json_path}")

    print(f"✅ {dataset_name} 정리 완료")

# 실행
clean_unmatched(train_image_dir, train_label_dir, train_json_dir, "학습 (Train)")
clean_unmatched(val_image_dir, val_label_dir, val_json_dir, "검증 (Val)")



🧹 학습 (Train) 데이터셋 정리 시작
🗑️ 이미지 삭제: /content/YOLO_safety/train/images/S2-N1106M17502.jpg
🗑️ JSON 삭제: /content/YOLO_safety/train/labels_json/S2-N1106M17502.json
🗑️ 이미지 삭제: /content/YOLO_safety/train/images/S2-N1104M00873.jpg
🗑️ JSON 삭제: /content/YOLO_safety/train/labels_json/S2-N1104M00873.json
🗑️ 이미지 삭제: /content/YOLO_safety/train/images/S2-N1105M06600.jpg
🗑️ JSON 삭제: /content/YOLO_safety/train/labels_json/S2-N1105M06600.json
🗑️ 이미지 삭제: /content/YOLO_safety/train/images/S2-O1301M01609.jpg
🗑️ JSON 삭제: /content/YOLO_safety/train/labels_json/S2-O1301M01609.json
🗑️ 이미지 삭제: /content/YOLO_safety/train/images/S2-N1106M16468.jpg
🗑️ JSON 삭제: /content/YOLO_safety/train/labels_json/S2-N1106M16468.json
🗑️ 이미지 삭제: /content/YOLO_safety/train/images/S2-N1105M04862.jpg
🗑️ JSON 삭제: /content/YOLO_safety/train/labels_json/S2-N1105M04862.json
🗑️ 이미지 삭제: /content/YOLO_safety/train/images/S2-N1106M13767.jpg
🗑️ JSON 삭제: /content/YOLO_safety/train/labels_json/S2-N1106M13767.json
🗑️ 이미지 삭제: /content/YOLO_safet

In [15]:
import os
from collections import Counter

def count_yolo_classes(label_dir, class_names=None):
    """
    YOLO 라벨(.txt) 파일에서 클래스별 인스턴스 수를 카운트합니다.

    :param label_dir: 라벨 파일(.txt)들이 있는 디렉토리
    :param class_names: 클래스 이름 리스트 (있으면 이름과 함께 출력됨)
    """
    counter = Counter()

    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(label_dir, filename), "r", encoding="utf-8") as f:
                for line in f:
                    class_id = line.strip().split()[0]  # 첫 번째 값이 class_id
                    counter[int(class_id)] += 1

    print("📊 클래스별 인스턴스 수:")
    for class_id, count in sorted(counter.items()):
        name = class_names[class_id] if class_names else f"class_{class_id}"
        print(f"  {name} ({class_id}): {count}개")

    return counter

label_dir = "/content/YOLO_safety/train/labels"  # 또는 val
class_names = ["safety_harness", "safety_helmet"]
count_yolo_classes(label_dir, class_names)

📊 클래스별 인스턴스 수:
  safety_harness (0): 50498개
  safety_helmet (1): 73563개


Counter({0: 50498, 1: 73563})

# 이미지 경로 평탄화

In [16]:
import shutil
from pathlib import Path

def flatten_image_dir_skip_duplicates(img_base_dir: Path):
    flat_img_dir = img_base_dir

    for subdir in img_base_dir.iterdir():
        if subdir.is_dir():
            for img_file in subdir.glob("*.jpg"):
                target_path = flat_img_dir / img_file.name

                if target_path.exists():
                    print(f"⚠️ 중복으로 건너뜀: {img_file.name}")
                    continue

                shutil.move(str(img_file), str(target_path))

    print(f"✅ 중복 건너뛰며 평탄화 완료: {img_base_dir}")

# 적용
flatten_image_dir_skip_duplicates(Path("/content/YOLO_safety/train/images"))
flatten_image_dir_skip_duplicates(Path("/content/YOLO_safety/val/images"))

✅ 중복 건너뛰며 평탄화 완료: /content/YOLO_safety/train/images
✅ 중복 건너뛰며 평탄화 완료: /content/YOLO_safety/val/images


# 데이터 존재 확인

In [17]:
from pathlib import Path

# 이미지 디렉토리 존재 확인
print("✅ train images exists:", Path("/content/YOLO_safety/train/images").exists())
print("✅ val images exists:", Path("/content/YOLO_safety/val/images").exists())

# 라벨 디렉토리 + 파일 개수 확인
val_labels = list(Path("/content/YOLO_safety/val/labels").glob("*.txt"))
print(f"✅ val 라벨 수: {len(val_labels)}개")

# 비어있는 파일 확인
empty = [f for f in val_labels if f.stat().st_size == 0]
print(f"⚠️ 비어있는 라벨 파일 수: {len(empty)}개")

✅ train images exists: True
✅ val images exists: True
✅ val 라벨 수: 10820개
⚠️ 비어있는 라벨 파일 수: 0개


In [18]:
# 잘못된 클래스 ID가 있는지 확인
bad_lines = 0
for f in val_labels:
    lines = f.read_text().splitlines()
    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue
        try:
            class_id = int(parts[0])
            if class_id not in [0, 1, 2]:
                print(f"⚠️ 잘못된 클래스 ID: {class_id} in {f.name}")
                bad_lines += 1
        except:
            print(f"⚠️ 형식 오류 in {f.name}: {line}")
            bad_lines += 1

print(f"❗ 잘못된 라벨 줄 수: {bad_lines}")

❗ 잘못된 라벨 줄 수: 0


In [19]:
# 라벨 파일 몇 개 출력해보기 (예: 처음 3개)
for f in val_labels[:3]:
    print(f"\n📄 파일명: {f.name}")
    print(f.read_text())


📄 파일명: s2-n1402m04638.txt
0 0.516667 0.649074 0.035417 0.083333
1 0.514062 0.583333 0.021875 0.042593

📄 파일명: s2-o1301m03746.txt
1 0.249219 0.481944 0.112500 0.261111
0 0.237109 0.772222 0.207031 0.455556

📄 파일명: s2-n1106m11241.txt
1 0.739062 0.221296 0.096875 0.235185
1 0.623958 0.368981 0.060417 0.125000
0 0.649479 0.561111 0.105208 0.277778
0 0.768490 0.569444 0.245312 0.507407


In [20]:
val_img_dir = Path("/content/YOLO_safety/val/images")
val_images = [f.stem for f in val_img_dir.glob("*.jpg")]
val_labels = [f.stem for f in Path("/content/YOLO_safety/val/labels").glob("*.txt")]

no_label_images = set(val_images) - set(val_labels)

print(f"⚠️ 라벨이 없는 이미지 수: {len(no_label_images)}")
print(list(no_label_images)[:5])  # 일부 예시

⚠️ 라벨이 없는 이미지 수: 0
[]


In [21]:
import os

# 존재 확인
print("val 이미지 디렉토리 있음:", os.path.exists("/content/YOLO_safety/val/images"))
print("val 라벨 디렉토리 있음:", os.path.exists("/content/YOLO_safety/val/labels"))

val 이미지 디렉토리 있음: True
val 라벨 디렉토리 있음: True


# 데이터 증강

In [22]:
import sys
sys.path.append('/content/drive/MyDrive')
from augmentations import apply_custom_augmentations

In [23]:
# /content/augmentations.py
import cv2
import numpy as np
import random

def apply_custom_augmentations(im, hyp):
    # Grayscale
    if hyp.get('grayscale', 0) > 0 and random.random() < hyp['grayscale']:
        im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)

    # Brightness
    if hyp.get('brightness', 0) > 0 and random.random() < hyp['brightness']:
        factor = 1.0 + np.random.uniform(-0.2, 0.2)
        im = np.clip(im * factor, 0, 255).astype(np.uint8)

    # Gamma correction
    if hyp.get('gamma', 0) > 0 and random.random() < hyp['gamma']:
        gamma = np.random.uniform(0.7, 1.5)
        inv_gamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** inv_gamma) * 255
                          for i in np.arange(256)]).astype("uint8")
        im = cv2.LUT(im, table)

    # CLAHE
    if hyp.get('clahe', 0) > 0 and random.random() < hyp['clahe']:
        lab = cv2.cvtColor(im, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        merged = cv2.merge((cl, a, b))
        im = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

    # Blur
    if hyp.get('blur', 0) > 0 and random.random() < hyp['blur']:
        k = random.choice([3, 5])  # 3x3 or 5x5 kernel
        im = cv2.GaussianBlur(im, (k, k), 0)

    # Gaussian Noise
    if hyp.get('noise', 0) > 0 and random.random() < hyp['noise']:
        noise = np.random.normal(0, 25, im.shape).astype(np.int16)
        im = np.clip(im.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    return im


In [24]:
hyp_text = """
# /content/YOLO_safety/hyp_cctv.yaml

# 증강 - 좌우/상하 반전
fliplr: 0.3
flipud: 0.0

# 증강 - Mosaic & Mixup
mosaic: 0.6

# 증강 - 색상 계열 (HSV)
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4

# 증강 - 기하학 왜곡
translate: 0.2
scale: 0.2
"""
with open("/content/YOLO_safety/hyp_cctv.yaml", "w") as f:
    f.write(hyp_text)

print("hyp_cctv.yaml 저장 완료")


hyp_cctv.yaml 저장 완료


# 학습

In [25]:
yaml_text = """
path: /content/YOLO_safety
train: train/images
val: val/images

names:
  0: safety_harness
  1: safety_helmet
"""

with open("/content/YOLO_safety/safety.yaml", "w") as f:
    f.write(yaml_text)

print("safety.yaml 저장 완료")

safety.yaml 저장 완료


## 증강 적용 X

In [ ]:
# from ultralytics import YOLO
# model = YOLO("yolov8n.pt")  # 또는 yolov8s.pt -> yolov8n.pt -> yolov8m.pt

# model.train(
#     data="/content/YOLO_safety/safety.yaml",            # 데이터셋 구성정보가 담긴 yaml
#     epochs=50,                                          # 전체 학습 반복 횟수
#     imgsz=640,                                          # 학습에 사용할 입력 이미지 크기
#     project="/content/drive/MyDrive/YOLO_safety/runs",  # 실험 결과 저장 경로
#     name="safetyGear_detect_exp1"                       # 실험명
# )


## 증강 적용 O

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive')
from augmentations import apply_custom_augmentations

In [26]:
import yaml
# hyp.yaml을 딕셔너리로 로드
with open("/content/YOLO_safety/hyp_cctv.yaml", "r") as f:
    hyp_dict = yaml.safe_load(f)

In [ ]:
from ultralytics import YOLO
import yaml, cv2, torch
from augmentations import apply_custom_augmentations

# 증강 하이퍼파라미터 (외부 증강용)
custom_hyp = {
    "grayscale": 0.05,
    "brightness": 0.1,
    "gamma": 0.05,
    "clahe": 0.05,
    "blur": 0.1,
    "noise": 0.1,
}

# YOLO 내부 증강 하이퍼파라미터 로드
with open("/content/YOLO_safety/hyp_cctv.yaml", "r") as f:
    yolo_hyp = yaml.safe_load(f)

# 모델 로드
model = YOLO("yolov8m.pt")

#  외부 증강 콜백 정의
def custom_callback(trainer):
    # train 단계에서만 적용
    if trainer.mode != "train":
        return

    images = trainer.batch[0]
    for i in range(len(images)):
        im = images[i].permute(1,2,0).cpu().numpy()
        im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)

        # 외부 증강 적용
        im = apply_custom_augmentations(im, custom_hyp)

        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        images[i] = torch.from_numpy(im).permute(2,0,1)

# 콜백 등록
model.add_callback("on_batch_start", custom_callback)

# 모델 학습
model.train(
    data="/content/YOLO_safety/safety.yaml",
    epochs=50,
    imgsz=960,
    project="/content/drive/MyDrive/YOLO_safety/runs",
    name="yolov8m_model_finished",
    optimizer="AdamW",
    cos_lr=True,
    patience=10,
    resume=False,
    save_period=5,
    **yolo_hyp  # YOLO 내부 증강
)


Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/YOLO_safety/safety.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.3, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.6, multi_scale=False, name=yolov8m_model_finished2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective=0.0,

train: Scanning /content/YOLO_safety/train/labels.cache... 43269 images, 0 backgrounds, 15 corrupt: 100%|██████████| 43284/43284 [00:00<?, ?it/s]

train: /content/YOLO_safety/train/images/s2-n1104m01723.jpg: ignoring corrupt image/label: image file is truncated (7 bytes not processed)
train: /content/YOLO_safety/train/images/s2-n1105m00865.jpg: ignoring corrupt image/label: image file is truncated (22 bytes not processed)
train: /content/YOLO_safety/train/images/s2-n1105m18951.jpg: ignoring corrupt image/label: image file is truncated (72 bytes not processed)
train: /content/YOLO_safety/train/images/s2-n1106m01094.jpg: ignoring corrupt image/label: image file is truncated (6 bytes not processed)
train: /content/YOLO_safety/train/images/s2-n1106m02923.jpg: ignoring corrupt image/label: image file is truncated (3 bytes not processed)
train: /content/YOLO_safety/train/images/s2-n1106m03577.jpg: ignoring corrupt image/label: cannot identify image file '/content/YOLO_safety/train/images/s2-n1106m03577.jpg'
train: /content/YOLO_safety/train/images/s2-n1106m06353.jpg: ignoring corrupt image/label: image file is truncated (78 bytes not p

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1466.8±879.9 MB/s, size: 619.0 KB)


val: Scanning /content/YOLO_safety/val/labels.cache... 10816 images, 0 backgrounds, 4 corrupt: 100%|██████████| 10820/10820 [00:00<?, ?it/s]

...

      12/50      16.2G      1.142     0.5173      1.251         24        960: 100%|██████████| 2705/2705 [10:12<00:00,  4.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 338/338 [00:56<00:00,  5.94it/s]


                   all      10816      30777      0.944      0.939      0.976      0.667

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      13/50      16.2G      1.133     0.5097      1.246         20        960: 100%|██████████| 2705/2705 [10:12<00:00,  4.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 338/338 [00:56<00:00,  5.97it/s]


                   all      10816      30777      0.943      0.942      0.977       0.67

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      14/50      16.2G      1.123     0.5011      1.244         70        960:  87%|████████▋ | 2358/2705 [08:54<01:21,  4.27it/s]

In [ ]:
# from ultralytics import YOLO

# model = YOLO("/content/drive/MyDrive/YOLO_safety/runs/Yolov8m_model_1747/weights/last.pt")

# # 학습 실행
# model.train(
#     data="/content/YOLO_safety/safety.yaml",               # 데이터셋 구성 정보
#     epochs=50,                                             # 전체 epoch 수
#     imgsz=960,                                              # 입력 이미지 크기
#     project="/content/drive/MyDrive/YOLO_safety/runs",     # 결과 저장 경로
#     name="Yolov8m_model_many",                              # 실험 이름

#     # 성능 향상을 위한 사용자 지정 값
#     optimizer='AdamW',       # 더 나은 수렴 속도와 일반화
#     cos_lr=True,             # 부드러운 cosine 학습률 스케줄링
#     patience=20,             # EarlyStopping 인내심
#     seed=42,                 # 재현성 확보
#     save_period=5,          # 10 epoch마다 중간 모델 저장

#     **hyp_dict               # 사용자 정의 하이퍼파라미터 (flip, hsv 등)
# )